In [1]:
import requests
from io import BytesIO 
from io import StringIO 
import gzip
import json
from datetime import datetime, date
import pyodbc
import ipaddress

In [2]:
url = 'https://snap.datastream.center/techquest'
file_name_template = '{report}-{date_file_name).{format).gz'
report = 'input'
format_name = 'json'
date_file_name = '2017-02-01'
resolution = 'gz'

server_name = 'DESKTOP-C5C3G74'
database_name = 'INV'
driver = 'SQL Server Native Client 11.0'

In [3]:
file_name = f'{report}-{date_file_name}.{format_name}.{resolution}'
full_url = url + '/' + file_name

In [4]:
response = requests.get(full_url)
if response.status_code==200:
    with gzip.open(BytesIO(response.content)) as f_in:
        try:
            file = f_in.read().decode('utf-8')
            cnxn = pyodbc.connect(f"Driver={driver};"
                          f"Server={server_name};"
                          f"Database={database_name};"
                          "Trusted_Connection=yes;")
            cursor = cnxn.cursor()
            api_data = date(int(date_file_name.split('-')[0]), int(date_file_name.split('-')[1]), 
                                                                                        int(date_file_name.split('-')[2]))
            for line in file.split('\n'):
                try:
                    json_line = json.loads(line)
                    user_id = int(json_line['user'])
                    ts = datetime.fromtimestamp(json_line['ts'])
                    ipaddress.ip_address(json_line['ip'])
                    ip = str(ipaddress.ip_address(json_line['ip']))
                    context = json.dumps((json_line['context']))
                    cursor.execute("insert into report_input(id_user, ts, context, ip)"
                                  "values (?, ?, ?, ?)",    (user_id, ts, context, ip))
                except Exception as ex:
                    cursor.execute("insert into data_error(api_report, api_data, row_text, error_text, ins_ts)" 
                                "values (?, ?, ?, ?, ?)", (report, api_data, line, str(ex), datetime.now()))
            cursor.commit()
        except Exception as ex:
                print(ex)      
else:
    print("Invalid requests")